# Data Generation

In [9]:
import numpy as np
import pandas as pd 
import queue
import random
import datetime

In [10]:
number_of_servers = 200
connections = 2000

In [11]:
server_matrix = np.zeros((number_of_servers,number_of_servers), dtype=int)

# Ensure that every server has at least on connection 
for i in range(1,number_of_servers):
    row = np.random.randint(1,number_of_servers-1)
    col = np.random.randint(1,number_of_servers)
    while row == i:
         row = np.random.randint(1,number_of_servers-1)
    while col == i:
        col = np.random.randint(1,number_of_servers-1)
    if i != number_of_servers-1:
        server_matrix[i,col] = 1
    server_matrix[row,i] = 1 

for i in range(connections):
    row = 0
    col = 0 
    while row == col:
        row = np.random.randint(1,number_of_servers-1)
        col = np.random.randint(1,number_of_servers)
    
    server_matrix[row,col] += 1 

server_matrix[0,np.random.randint(1,number_of_servers)-1] =1 
print(server_matrix)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 1 0 ... 0 0 1]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [12]:
max_server_number = np.amax(server_matrix, axis=0)
print(max_server_number)

[0 2 1 1 2 2 1 2 1 2 2 2 2 1 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 2 1 1 2 1 1
 2 1 2 2 1 1 1 1 1 2 1 2 1 1 1 1 2 1 1 2 1 2 1 1 2 1 2 1 2 1 2 1 1 1 2 2 1
 1 1 1 1 1 1 2 1 1 1 1 1 2 2 1 1 1 1 1 1 2 2 1 2 1 1 2 1 2 2 2 2 2 3 1 1 2
 1 1 1 2 1 1 1 2 2 1 3 1 1 2 2 1 1 1 2 2 1 1 1 2 1 1 2 2 1 2 2 1 1 1 1 1 1
 2 2 1 1 2 2 1 1 3 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 2 2 1 2 2 2 1 1 1 1
 1 1 1 2 1 1 1 1 1 1 2 1 2 1 1]


In [13]:
server_names = []
count = 0

for max_number in max_server_number:
    if count == 0:
        server_names.append("null")
    elif max_number == 1:
        server_names.append("S-" + str(count))
    elif max_number > 1:
        tmp = []
        for i in range(1, max_number+1):
            tmp.append("S-" + str(count)+ "." + str(i))
        server_names.append(tmp)
    else:
        continue
    count += 1


In [14]:
print(len(server_names))

200


In [16]:
number_cases = 10000
req = ["Request", "Response"]
max_requests = 5

with open("data.txt", "w") as file:
    for i in range(number_cases):
        
        count_requests = 0 
        q_servers = queue.LifoQueue()
        q_server_numbers = queue.LifoQueue()

        q_servers.put("null")
        q_server_numbers.put(0)

        server_number = np.random.choice(np.nonzero(server_matrix[0])[0])
        q_server_numbers.put(server_number)

        if max_server_number[server_number]==1:
            server = np.random.choice([server_names[server_number]])
        else:
            server = np.random.choice(server_names[server_number])
        q_servers.put(server)

        hours = random.randint(0, 23)
        minutes = random.randint(0, 59)
        seconds = random.randint(0, 59)
        year = 2023
        month = random.randint(1,12)
        day = random.randint(1,28)
        process_time = datetime.datetime(year=year,month=month,day=day,hour=hours, minute=minutes, second=seconds)
        
        file.write("<null, "+ server + ", "+ str(process_time) + ", Request, " + str(i) +">\n")

        count_requests += 1
        while count_requests <= max_requests+1:
            if q_servers.queue[-1] == 'null' and count_requests>=1:
                break
            r = np.random.choice(req)
            if r == "Response":
                process_time = process_time + datetime.timedelta(minutes=random.randrange(60))
                file.write("<"+ q_servers.queue[-1] +", " + q_servers.queue[-2] + ", "+ str(process_time) + ", Response, " + str(i) +">\n")
                q_servers.get()
                q_server_numbers.get()

            else:
                if np.nonzero(server_matrix[q_server_numbers.queue[-1]])[0].size == 0:
                    continue
                req_server_number =  np.random.choice(np.nonzero(server_matrix[q_server_numbers.queue[-1]])[0]) - 1 
                count_requests += 1
                if len(np.nonzero(server_matrix[q_server_numbers.queue[-1]])[0])==1 and req_server_number in q_server_numbers.queue:
                    continue
                while req_server_number in q_server_numbers.queue:
                    req_server_number =  np.random.choice(np.nonzero(server_matrix[q_server_numbers.queue[-1]])[0])
                q_server_numbers.put(req_server_number)
                
                if max_server_number[req_server_number]==1:
                    server = np.random.choice([server_names[req_server_number]])
                else:
                    server = np.random.choice(server_names[req_server_number])

                q_servers.put(server)
                process_time = process_time + datetime.timedelta(minutes=random.randrange(60))
                file.write("<" + q_servers.queue[-2] + ", " + server + ", "+ str(process_time) + ", Request, " +str(i)+">\n")
        
        while  not q_servers.empty():
            if q_servers.queue[-1] == 'null':
                break
            else:
                process_time = process_time + datetime.timedelta(minutes=random.randrange(60))
                file.write("<"+ q_servers.queue[-1] +", " + q_servers.queue[-2] + ", "+ str(process_time) + ", Response, " + str(i) +">\n")
                q_servers.get()